In [22]:
# initializing and importing modules

import requests
import pprint
import requests
import json
import webbrowser # webbrowser.open(url)
from typing import Any, Text, Dict, List
from rasa_sdk import Action, Tracker
from rasa_sdk.executor import CollectingDispatcher

currency = "USD"
headers = {
        'x-rapidapi-key': "cf03cdb80emshcfe16b58ab7e55ap17a505jsn17df98d33df1",
        'x-rapidapi-host': "booking-com.p.rapidapi.com"
}
pp = pprint.PrettyPrinter()
prototype_mode = 1

In [24]:
class DestinationSelection(Action):
        def name(self) -> Text:
            return "action_destination_selection"

        def run(self, dispatcher: CollectingDispatcher,
                tracker: Tracker,
                domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:
            
            
            
            buttons = [
                {"payload":'/ok{"intent_button":"faq-portal"}',"title":"Portal"},
            ]

            dispatcher.utter_message(text="Which one of these destinations do you mean?", buttons=buttons)

            return []

In [20]:
class HotelSelection(Action):
        def name(self) -> Text:
            return "action_hotel_selection"

        def run(self, dispatcher: CollectingDispatcher,
                tracker: Tracker,
                domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:
            
            dest_type = destinations_selection_dict[dest_id][1]
            url = "https://booking-com.p.rapidapi.com/v1/hotels/search"

            querystring = {"units":"metric",
                           "checkin_date":check_in,
                           "checkout_date":check_out,
                           "filter_by_currency": currency,
                           "adults_number":number_of_people,
                           "room_number":number_of_rooms,
                           "dest_id":dest_id,
                           "locale":"en-us",
                           "dest_type":dest_type,
                           "order_by":order_by
                           }

            response = requests.request("GET", url, headers=headers, params=querystring)

            json = response.json()

            hotels_selection_dict = {}
            for i in range(0, 5):
                hotels_selection_dict[json["result"][i]["hotel_id"]] =  [json["result"][i]["hotel_name"], 
                                              str(round(json["result"][i]["min_total_price"], 2)) + " " + json["result"][i]["price_breakdown"]["currency"], 
                                              json["result"][i]["url"], json["result"][i]["main_photo_url"]]
            buttons = [
                {"payload":'/ok{"intent_button":"faq-portal"}',"title":"Portal"},
            ]

            dispatcher.utter_message(text="Hotels:", buttons=buttons)

            return []

In [21]:
class ItinerarySelection(Action):
        def name(self) -> Text:
            return "action_itinerary_selection"

        def run(self, dispatcher: CollectingDispatcher,
                tracker: Tracker,
                domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:
            
            url = "https://booking-com.p.rapidapi.com/v1/hotels/nearby-places"

            querystring = {"locale":"en-us","hotel_id":hotel_id}

            response = requests.request("GET", url, headers=headers, params=querystring)

            json = response.json()

            response = requests.request("GET", url, headers=headers, params=querystring)

            json = response.json()

            itinerary_closests, itinerary_populars = [], []
            for i in range(0, len(json['landmarks']['closests'])):
                itinerary_closests.append(json['landmarks']['closests'][i].get('landmark_name'))
            for i in range(0, len(json['landmarks']['populars'])):
                itinerary_populars.append(json['landmarks']['populars'][i].get('landmark_name'))
            
            buttons = [
                {"payload":'/ok{"intent_button":"faq-portal"}',"title":"Portal"},
            ]

            dispatcher.utter_message(text="Near your hotel you can find:", buttons=buttons)

            return []